## Fish Class

In [186]:
import matplotlib.pyplot  as plt 
import scipy.stats as st
import numpy as np
import PyQt5
import math

%matplotlib qt

In [187]:
class Bird():
    def __init__(self,id,area=(100,100),speed=0.5) -> None:
   
        
        self.pos = [0,0]
        self.pos[0]= st.uniform(0,area[0]).rvs()
        self.pos[1]= st.uniform(0,area[1]).rvs()
        self.area =area
        self.id=id
        self.speed= speed
        self.dist_speed=st.norm(0,self.speed)
        self.alpha=0.5
        self.temp_posX=self.pos[0]
        self.temp_posY=self.pos[1]
        self.offset=0
    def draw(self):
        
        plt.plot(self.pos[0],self.pos[1],'ro')
        # plt.text(self.pos[0]+1,self.pos[1]+1,self.id)
        # plt.arrow(self.temp_posX, self.temp_posY, self.pos[0]-self.temp_posX, self.pos[1]-self.temp_posY)


    def move(self,):
        self.dist_move=(st.norm(0,abs(self.dist_speed.rvs())))

        self.pos[0]+=(self.dist_move.rvs())
        self.pos[1]+=(self.dist_move.rvs())

        self.pos=(list(map(self._clip,self.pos,[0,0],self.area)))
    
    

    def move_to_neighbor(self,neighbor):
        vector_to_neighbor = [neighbor.pos[0] - neighbor.temp_posX, neighbor.pos[1] - neighbor.temp_posY]
        angle = math.radians(st.norm(loc=10, scale=1).rvs())
        
        rotated_vector = [
            vector_to_neighbor[0] * math.cos(angle) - vector_to_neighbor[1] * math.sin(angle),
            vector_to_neighbor[0] * math.sin(angle) + vector_to_neighbor[1] * math.cos(angle)
        ]
        # Update the position based on the rotated vector
        self.temp_posX = self.pos[0]
        self.temp_posY = self.pos[1]
        self.pos[0] += self.alpha * rotated_vector[0]
        self.pos[1] += self.alpha * rotated_vector[1]

        
        self.pos=(list(map(self._clip,self.pos,(0,0),self.area)))
    def avoid_neighbor(self, neighbor):
    # Calculate vector from current position to neighbor's position
        vector_to_neighbor = [neighbor.pos[0] - self.pos[0], neighbor.pos[1] - self.pos[1]]
        
        # Calculate angle for 90-degree rotation (in radians)
        direction=st.randint(0,2).rvs()
        if direction== 0:
            angle = math.radians(st.norm(loc=90, scale=1).rvs())
        else :
            angle = math.radians(st.norm(loc=-90, scale=1).rvs())
        
        # Rotate the vector by 90,-90 degrees
        rotated_vector = [
            vector_to_neighbor[0] * math.cos(angle) - vector_to_neighbor[1] * math.sin(angle),
            vector_to_neighbor[0] * math.sin(angle) + vector_to_neighbor[1] * math.cos(angle)
        ]
        # Update the position based on the rotated vector
        self.temp_posX = self.pos[0]
        self.temp_posY = self.pos[1]
        self.pos[0] += self.alpha * rotated_vector[0]
        self.pos[1] += self.alpha * rotated_vector[1]
        
        # Clip the position to the valid range
        self.pos = list(map(self._clip, self.pos, (0, 0), self.area))
        

    @staticmethod  #clip a value between a & b
    # We used static method because it's not related to each instance calling it but general to the class
    def _clip(value, a, b):
        return min(max(value, a), b) # to not go beyond the area


In [188]:
print(st.randint(0,2).rvs())

0


## Main()

In [189]:
def compute_adj(birds,nb):
    adj_matrix=np.zeros((nb,nb))

    for i in range(nb):
        for j in range(nb):
            if i == j :
                adj_matrix[i][j]=0
            else:
                dist = np.linalg.norm(np.array(birds[i].pos) - np.array(birds[j].pos))
                
                adj_matrix[i][j]=dist
                

    return adj_matrix

def calculate_angle(point1, point2, angle_offset):# this fucntion compute angle between two birds
    delta_x = point2[0] - point1[0]
    delta_y = point2[1] - point1[1]
    
    # Calculate the angle in radians
    angle_rad = np.arctan2(delta_y, delta_x)
    
    # Convert the angle to degrees and add the offset
    angle_deg = np.degrees(angle_rad) - angle_offset
   
    return angle_deg % 360


angular_range=80
offset=0
def compute_neighbors(birds,reference_fish,adj_matrix,nb):# This fucntion return all infomations aobut neighbors
    neighbors=[]
    angle=calculate_angle((reference_fish.temp_posX, reference_fish.temp_posY), (reference_fish.pos[0], reference_fish.pos[1]),reference_fish.offset)
    reference_fish.offset=(reference_fish.offset+angle)% 360
    
    for i, other_fish in enumerate(birds):
        if other_fish==reference_fish: continue
        angle = calculate_angle(reference_fish.pos, other_fish.pos,reference_fish.offset-(angular_range/2))# fish_offset minus angular range divided by two to get the new angle in terms of the angular range
        dist =adj_matrix[reference_fish.id][other_fish.id]
        if 0 <= angle <= angular_range and dist>0 :
            neighbors.append([reference_fish.id,other_fish.id,angle,dist])
    
    return neighbors
            

def neighbor_in_front(neighbors,nb_birds):# this function the neighbor in front for all birds
    front_neighbor=np.zeros(nb_birds,'i')
    avoid_neighbors=np.zeros(nb_birds,'i')
    for i in range (len(neighbors)):
        k=angular_range 
        changed = 0
        for j in range (len(neighbors[i])):   
            angle =abs(neighbors[i][j][2] - angular_range/2)# this variable represent the diffrence between the angle with the neighbor an the center vision 
            if angle < k: # if it is lower than the previous so we update it
                front_neighbor[i] = neighbors[i][j][1]
                changed =1
                k=angle
                
                if neighbors[i][j][3]<3:# too close so it doesn't count it as a neighbor infront 
                    front_neighbor[i]=-2
                    avoid_neighbors[i]=neighbors[i][j][1]
        if changed == 0 : # if it didn't find any neighbor in his vision
            front_neighbor[i] =-1
    return [front_neighbor,avoid_neighbors]


def destination_decision(neighbors,nb_birds,i):
    front_neighbor,avoid_neighbors=neighbor_in_front(neighbors,nb_birds)
    if front_neighbor[i] == - 1:# if the fish hasn't any neighbor or it is too close
        return [-1,0]
    elif front_neighbor[i] == - 2:# if the fish hasn't any neighbor or it is too close
        return [-2,avoid_neighbors[i]]
    else:

        initial_probabilities = np.full(nb_birds, 1/nb_birds) # compute probability
        initial_probabilities[front_neighbor[i]] = 4 # give high chance to neighbor infront
        probabilities = initial_probabilities / np.sum(initial_probabilities)# normalizing

    outcomes = np.arange(0,nb_birds)

    custom_distribution = st.rv_discrete(values=(outcomes, probabilities))

    simulations = 10
    results = custom_distribution.rvs(size=1)[0]
    results
    return [1,results]
        


In [190]:



time_steps = 300
nb_birds =20
area=(100, 100)
birds=[]

for i in range(nb_birds):
    birds.append(Bird(i,area))

for t in range(time_steps):
    neighbors=[]


    adj_matrix=compute_adj(birds,nb_birds)
    
    
    for i in range(nb_birds):
        neighbors.append(compute_neighbors(birds,birds[i],adj_matrix,nb_birds))   # return neighbors of each fish in its index
    for i in range(nb_birds):

        destination = destination_decision(neighbors,nb_birds,i)

        if destination[0]== -2:
            birds[i].avoid_neighbor(birds[destination[1]])
        if destination[0]== -1:
            birds[i].move()
        else:
            birds[i].move_to_neighbor(birds[destination[1]])

        
    
    plt.clf() #clear the figure
    for i in range(nb_birds):
        birds[i].draw()
        plt.text(2,2,f'time step: {t}')
    plt.axis([0, 100, 0, 100]) #fix draw axis to 10x10
    plt.pause(0.02)  # Update the drawing

